In [1]:
import requests
import html
import ast
import time
import pprint
import pandas as pd
import datetime
import csv
from bs4 import BeautifulSoup
url = "https://www.canberrahealthservices.act.gov.au/_designs/generic-modules/entitywaitingtimes/asset-listings/dynamic-wait-times-data-raw-asset-listing/_nocache"

In [2]:
curr_time = str(datetime.datetime.now())
filename = 'chs_wait_times_'+curr_time+'.csv'

In [ ]:
header_written = False
try:
    while True:
        # Get current wait times as dictionary
        response = requests.get(url) # HTTP request
        soup = BeautifulSoup(response.text,'html.parser') # Create parser
        data_element = soup.find('div',id='divEntitiesJSON').get_text(strip=True) # Get element from page
        to_dict = ast.literal_eval(data_element) # Convert to dictionary
        
        # Convert to single data row
        rows = []
        for entry in to_dict:
            id_ = entry['id']
            last_updated = entry['lastUpdated']
            clinic_type = entry['type']
            queue = next((x['queue'] for x in entry['waitTimes'] if x['name'] == 'Wait'), None)
            wait_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Wait'), None)
            treat_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Treat'), None)
            total_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Total'), None)
            to_append = {
                'id':id_,
                'lastUpdated':last_updated,
                'clinic_type':clinic_type,
                'queue':queue,
                'wait_time':wait_time,
                'treat_time':treat_time,
                'total_time':total_time
            }
            rows.append(to_append)
        df = pd.DataFrame(rows)
        with open(filename,mode='a',newline='') as file:
            csv_writer = csv.writer(file)
            if not header_written:
                df.to_csv(file,header=True,index=False,mode='a')#,line_terminator='\n')
                header_written = True
            else:
                df.to_csv(file,header=False,index=False,mode='a')#,line_terminator='\n')
            #print('Successfully appended row, zzz')
        time.sleep(59)
except Exception as e:
    print(str(datetime.datetime.now()))
    raise(e)